## Build a riverine SFINCS model from scratch

This notebook demonstrates how to prepare **SFINCS** model from scratch using the command line interace (CLI).

All lines in this notebook which start with `!` are executed from the command line. Within the notebook environment the logging messages are shown after completion. You can also copy these lines and paste these in your shell to get more direct feedback.

### hydroMT CLI build interface

Lets first check if the SFINCS model is recognized by hydromt

In [1]:
# this should return "sfincs"
!hydromt --models

hydroMT model plugins: sfincs (v0.1.0)


Using the **hydromt build** API we can setup a complete model from scratch. Let's get an overview of all the available options:

In [2]:
!hydromt build --help

Usage: hydromt build [OPTIONS] MODEL MODEL_ROOT REGION

  Build models from source data.

  Example usage:
  --------------

  To build a wflow model for a subbasin using and point coordinates snapped to cells with stream order >= 4
  hydromt build wflow /path/to/model_root "{'subbasin': [-7.24, 62.09], 'strord': 4}" -i /path/to/wflow_config.ini

  To build a wflow model based on basin ID
  hydromt build wflow /path/to/model_root "{'basin': 230001006}"

  To build a sfincs model based on a bbox (for Texel)
  hydromt build sfincs /path/to/model_root "{'bbox': [4.6891,52.9750,4.9576,53.1994]}"

Options:
  -r, --res FLOAT             Model resolution in model src.
  --build-base / --build-all  Deprecated!
  --opt TEXT                  Component specific keyword arguments, see the
                              setup_<component> method of the specific model
                              for more information about the arguments.

  -i, --config PATH           Path to hydroMT configuration fi

### setup SFINCS model base layers 

In [3]:
# NOTE: copy this line (without !) to your shell for more direct feedback
!hydromt build sfincs "./sfincs_test_base" "{'bbox': [12.478552, 45.600758, 12.671882, 45.687984]}" -r 100 -vv



2021-05-10 12:11:50,622 - build - log - DEBUG - Writing log messages to new file /home/hcwinsemius/git/hydromt_sfincs/examples/sfincs_test_base/hydromt.log.
2021-05-10 12:11:50,622 - build - log - INFO - HydroMT version: 0.4.1.dev
2021-05-10 12:11:50,622 - build - main - INFO - Building instance of sfincs model at /home/hcwinsemius/git/hydromt_sfincs/examples/sfincs_test_base.
2021-05-10 12:11:50,622 - build - main - INFO - User settings:
2021-05-10 12:11:50,623 - build - model_api - INFO - Initializing sfincs model from hydromt_sfincs (v0.1.0).
2021-05-10 12:11:50,623 - build - model_api - WARNING - Model dir already exists and files might be overwritten: /home/hcwinsemius/git/hydromt_sfincs/examples/sfincs_test_base/.
2021-05-10 12:11:50,623 - build - model_api - DEBUG - setup_basemaps.region: {'bbox': [12.478552, 45.600758, 12.671882, 45.687984]}
2021-05-10 12:11:50,623 - build - model_api - DEBUG - setup_basemaps.res: 100.0
2021-05-10 12:11:50,623 - build - model_api - DEBUG - setu

The example above means the following: run **hydromt build** with:

* `sfincs` : i.e. build a sfincs model
* `./sfincs_test_base` : output model folder
* `"{'bbox': [45.600758, 12.478552, 45.687984,  12.671882]}"` : make a model that is active for the full given bounding box. All *REGION* options are described in the [docs](https://deltares.github.io/hydromt/latest/user_guide/cli.html#region-options)
* `-r 100` : build the model at a 100 meter resolution
* `-vv` : give some extra verbosity (2 * v) to display feedback on screen. Now debug messages are provided.

NOTE: As we did not specify a model configuration, only the base maps (topography) have been setup using default parameters. To build a complete model we need the use a configuraton ini-file.

### Model setup configuration

The ini-file contains the model setup configuration and determines which components are build and in which order and optionally sets non-default arguments for each component. This configuration is passed to hydromt using `-i <path_to_ini_file>`. We have prepared several example ini-files which are available in the model repository [examples folder](https://github.com/Deltares/hydromt_sfincs/tree/main/examples) and from the [docs(build_configuration)](https://deltares.github.io/hydromt_sfincs/latest/user_guide/sfincs/build_configuration.html). 

Each header as shown between `[...]` (e.g. `[setup_basemaps]`) corresponds to a model component. All model components are explained in the [docs(model_components)](https://deltares.github.io/hydromt_sfincs/latest/user_guide/sfincs/components.html). 

We will load the default SFINCS build ini file for inspection:

In [4]:
fn_ini = 'sfincs_riverine.ini'
with open(fn_ini, 'r') as f:
    txt = f.read()
print(txt)

[global]
write_gis = True                # write addtional geotif and geojson files to gis subfolder
config_fn = sfincs.inp          # name of sfincs config (inp) file

[setup_config]
tref = 20100101 000000
tstart = 20100101 000000
tstop = 20100103 000000
alpha = 0.5
zsini = 0.5

[setup_basemaps]
basemaps_fn = merit_hydro       # hydrography basemaps ['elevtn', 'flwdir', 'uparea', 'basins']
landmask_fn = osm_landareas     # land polygon
crs = utm                       # model CRS (must be UTM zone)
bathymetry_fn = gebco           # bathymetry ['elevtn']
mdt_fn = dtu10mdt_egm96         # mean dynamic topography ['mdt']
merge_buffer = 0                # buffer (cells) between elevation and bathymetry data filled with 'linear' interpolation
mask_fn = None                  # polygon to mask active model cells 
dep_contour = -1                # mimimum depth threshold for active model cells

[setup_cn_infiltration]
cn_fn = gcn250                  # curve number source
antecedent_runoff_cond

### Data for model setup

The `<...>_fn` arguments correspond to a data source from the `DataCatalog` which is based on a yml-file with references to the data paths/urls and how the data should be read. This file can be provided to hydromt using `-d <path_to_yml_file>`. 

By default some example data for the Piave basin will be downloaded to `~/.hydromt_data/` which is also used for this example. An overview of the available example data is provided [here](https://deltares.github.io/hydromt/latest/user_guide/data.html#available-global-datasets)
This example data is a based on the data which available from the Deltares p-drive. 
If you have acces to this drive, a pre-configured catalog file can be loaded using the `--dd` flag. 

More background how to write a data catalog yml file can be found in the [hydromt core docs](https://deltares.github.io/hydromt/latest/user_guide/data.html) 

### Setup complete SFINCS model 

In [5]:
# NOTE: copy this line (without !) to your shell for more direct feedback
!hydromt build sfincs "./sfincs_test_full" "{'bbox': [12.478552, 45.600758, 12.671882, 45.687984]}" -r 100 -i sfincs_riverine.ini -vv

2021-05-10 12:12:18,441 - build - log - DEBUG - Writing log messages to new file /home/hcwinsemius/git/hydromt_sfincs/examples/sfincs_test_full/hydromt.log.
2021-05-10 12:12:18,441 - build - log - INFO - HydroMT version: 0.4.1.dev
2021-05-10 12:12:18,441 - build - main - INFO - Building instance of sfincs model at /home/hcwinsemius/git/hydromt_sfincs/examples/sfincs_test_full.
2021-05-10 12:12:18,441 - build - main - INFO - User settings:
2021-05-10 12:12:18,483 - build - cli_utils - INFO - global.write_gis: True
2021-05-10 12:12:18,484 - build - cli_utils - INFO - global.config_fn: sfincs.inp
2021-05-10 12:12:18,484 - build - cli_utils - INFO - setup_config.tref: 20100101 000000
2021-05-10 12:12:18,484 - build - cli_utils - INFO - setup_config.tstart: 20100101 000000
2021-05-10 12:12:18,484 - build - cli_utils - INFO - setup_config.tstop: 20100103 000000
2021-05-10 12:12:18,484 - build - cli_utils - INFO - setup_config.alpha: 0.5
2021-05-10 12:12:18,484 - build - cli_utils - INFO - se

With this example we build a complete SFINCS model including forcing. Compared to the previous **hydromt build** we have added:
* `-i sfincs_riverine.ini` : setup configuration file including all components to build and their arguments

Next we check which files have been created. The model root should contain two netcdf (.nc) files, one for the staticmaps and one for the forcing data, a SFINCS configuration (sfincs.inp) file. These files are sufficient to run the SFINCS model. In addition several geometry (.geojson) files are saved in the *staticgeoms* folder.  Finally, the setup process is logged in the hydromt.log file. 

In [6]:
import os
root = 'sfincs_test_full'
for path, _, files in os.walk(root):
    print(path)
    for name in files:
        if name.endswith('.xml'):
            continue
        print(f' - {name}')

sfincs_test_full
 - sfincs.man
 - hydromt.log
 - sfincs.dep
 - sfincs.inp
 - sfincs.scs
 - sfincs.ind
 - sfincs.msk
sfincs_test_full/gis
 - region.geojson
 - bbox.geojson
 - msk.tif
 - manning.tif
 - dep.tif
 - scs.tif


### Visualize and/or inspect model schematization

* The **SFINCS plot riverine** example notebook contains scripts to visualize your model
